# Analysis of the AMLTK Pipeline
OpenFE-like Analysis

### To Do for the User

In [69]:
# TODO 1 for the user: Adapt list of labels (has to correspond to the method names for which the AMLTK was executed in the src/amtlk/main.py main()-method
label_list = ['Original Data', 'Self-feature-engineered Data', 'autofeat Data', 'OpenFE Data', 'H2O', 'MLJAR', 'AutoGluon']
# TODO 2 for the user: Adjust part size number according to the max_trials variable defined in src/amltk/main.py
part_size = 10
# TODO 3 for the user: Define the names of the used datasets
# all 10 datasets
datasets = ["abalone dataset", "moneyball dataset", "australian dataset", "blood transfusion service center dataset", "car dataset", "churn dataset", "credit g dataset", "kr-vs-kp dataset", "pc4 dataset", "wilt dataset"]#, "wine-quality-white dataset"]
# 5 small datasets
# datasets = ["australian dataset", "blood transfusion service center dataset", "car dataset", "credit g dataset", "wilt dataset"]
# 1 dataset to test new methods
# datasets = ['blood transfusion service center dataset']
# TODO 4 for the user. Define number of folds
folds = 1

### Imports


In [70]:
import pandas as pd
import numpy as np
from tabulate import tabulate

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Read Files

In [71]:
matrix = []

for dataset in datasets:
    df_dataset = pd.DataFrame()
    for fold in range(folds):
        dataset = dataset.replace(" ", "_")
        dataset = dataset.replace("-", "_")
        df_fold = pd.read_parquet("results_" + str(dataset) + "_fold_" + str(fold) + ".parquet")
        df_dataset = pd.concat([df_dataset, df_fold])
    df_dataset['index_col'] = np.arange(len(df_dataset))
    df_success = df_dataset[df_dataset["status"] != "fail"]
    total_parts = int(df_success.shape[0] / part_size)
    failed_trials = []
    for index in df_dataset['index_col']:
        if df_dataset["status"][index] == "fail":
            failed_trials.append(index)
    failed_trials = [int(x / part_size) for x in failed_trials]
    failed_trials = list(set(failed_trials))
    label_list_success = []
    for i in range(len(label_list)):
        label_list_success.append(str(label_list[i]))
    i = 0
    for elem in failed_trials:
        elem = elem - i
        label_list_success.pop(elem)
        i += 1
    metric_accuracy = df_success["metric:accuracy [0.0, 1.0] (maximize)"].tolist()
    metric_accuracy = [sum(metric_accuracy[x:x+part_size]) for x in range(0, len(metric_accuracy), part_size)]
    metric_accuracy.insert(0, dataset)
    print(metric_accuracy)
    matrix.append(metric_accuracy)

['abalone_dataset', 2.539019189765458, 2.522525291475752, 2.545399333121626, 2.4522926779476477, 2.4448526743183776, 2.517729551331489, 0.472740779385746]
['moneyball_dataset', 0.06947919924929621, 0.0821017099363987, 0.06952481493066417, 0.05409368157647795]
['australian_dataset', 8.650120712620712, 8.38838245088245, 8.65174408924409, 8.671599234099233, 8.587745587745587, 8.616487678987678, 8.70004995004995]
['blood_transfusion_service_center_dataset', 7.668522408963586, 7.6106092436974775, 7.634296218487394, 7.558350840336134, 7.571901260504202, 7.594152661064425, 7.668522408963586]
['car_dataset', 8.477917657943431, 7.9402788791964065, 9.518358445678032, 9.663663098070314, 9.333743061062648, 8.185867697594503]
['churn_dataset', 9.213124909135733, 8.958455275816513, 9.367345672964484, 9.530903728121464, 9.183559651207627, 9.322007958129745, 9.186449134972156]
['credit_g_dataset', 7.368441845764854, 7.337636298988622, 7.472927860303413, 7.481807048040455, 7.345132743362832, 7.48528365

In [73]:
table = tabulate(matrix, label_list, tablefmt="fancy_grid")
with open('table.txt', 'w') as outputfile:
    outputfile.write(table)
print(table)

╒══════════════════════════════════════════╤═════════════════╤════════════════════════════════╤═════════════════╤═══════════════╤═════════╤═════════╤═════════════╕
│                                          │   Original Data │   Self-feature-engineered Data │   autofeat Data │   OpenFE Data │     H2O │   MLJAR │   AutoGluon │
╞══════════════════════════════════════════╪═════════════════╪════════════════════════════════╪═════════════════╪═══════════════╪═════════╪═════════╪═════════════╡
│ abalone_dataset                          │       2.53902   │                      2.52253   │       2.5454    │     2.45229   │ 2.44485 │ 2.51773 │    0.472741 │
├──────────────────────────────────────────┼─────────────────┼────────────────────────────────┼─────────────────┼───────────────┼─────────┼─────────┼─────────────┤
│ moneyball_dataset                        │       0.0694792 │                      0.0821017 │       0.0695248 │     0.0540937 │         │         │             │
├───────────────